Iterative Solvers
===

So far we have used direct solvers to solve the linear system of equations. Although a direct solver can profit from the sparse matrix, it's arithmetic complexity is sub-optimal. For large-scale problems iterative solvers are a must.

The conjugate gradient (cg) method is the standard method for symmetric and positive definite matrices. It's convergence rate depends on a preconditioner, what is a cheap approximative inverse to the matrix.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw

We generate a 3D geometry and mesh using the constructive solid geometry (CSG) modeler of Netgen:

In [ ]:
from netgen.csg import *
geo = CSGeometry()
cube = OrthoBrick (Pnt(0,0,0), Pnt(1,1,1)).bc("outer")
cyl = Cylinder( Pnt(1,0.5,0.5), Pnt(0,0.5,0.5), 0.2).bc("inner")
geo.Add (cube-cyl, col=(0,0,1))
geo.Draw()

ngmesh = geo.GenerateMesh(maxh=0.1)
ngmesh.Refine()
ngmesh.Refine()
mesh = Mesh(ngmesh)

mesh.Curve(3)
Draw (mesh)

In [ ]:
fes = H1(mesh, order=3, dirichlet="outer", wb_withoutedges=True)
print ("we have", fes.ndof, "unknowns")
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx

f = LinearForm(fes)
f += v*dx

# c = Preconditioner(a, "direct", inverse="sparsecholesky")
# c = Preconditioner(a, "local")
c = Preconditioner(a, "bddc")

gfu = GridFunction(fes)

assemble system and setup preconditioner in parallel:

In [ ]:
ngsglobals.msg_level=5
with TaskManager():
    a.Assemble()
    f.Assemble()

solve the system using the preconditioned conjugate gradient method:

In [ ]:
from ngsolve.solvers import CG

with TaskManager():
    CG (mat=a.mat, pre=c.mat, rhs=f.vec, sol=gfu.vec, 
        printrates=True, maxsteps=200)

In [ ]:
Draw (gfu)